<img src=https://brand.uark.edu/_resources/images/UA_Logo_Horizontal.jpg width="400" height="96">

###_Artificial intelligence for image processing and analysis._

# Notebook 4.2 CNN Competition
---
##### The purpose of this notebook is to set up a competition using convolutional neural networks.



### Required packages
---
##### **_NOTE: This notebook will require the use of GPU hardware acceleration. please refer to notebook 2.4_ParallelProcessing if you need  refresher on how to do this._**
##### **_Run this code chunk first. If you encounter an error when trying to run code chunks in this notebook, then first try re-running this chunk._**


In [37]:
# Import all of the necessary packages
import numpy as np
import imageio
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as T
import time
import pandas as pd
from IPython.display import clear_output
import ipywidgets as widgets

# Faces dataset
---
##### The following code chunks need to be run to define the dataset class and download the dataset

In [38]:
class UTKFaceDataset(torch.utils.data.Dataset):
  # Define what will be ran at initialization of the UTKFaceDataset class
  def __init__(self, image_array, age_array):
    # Attach the list of images and ages to the class
    # These are attached to the class so that they can be accessed by other methods in the class
    self.images = image_array
    self.ages = age_array

    # Initialize a transform that will be used later
    self.tform = T.ToTensor()                       

  # There are two required methods for a class that inherits from torch.utils.data.Dataset:
  # __len__()
  # __getitem__()

  # Return the length of the dataset
  def __len__(self):
    return len(self.ages)

  # Return a single image from the dataset, as well as the age associated with the image
  def __getitem__(self,idx):
    # Return a single variable (dict) that contains both the image and the age
    out = {
        'image': self.tform(self.images[:,:,idx]).float(),
        'age': torch.tensor(self.ages[idx]).float()
    }
    return out

In [39]:
import os

# Check to see if file exists
filename = "/content/age_gender.csv"
if not os.path.isfile(filename):
  # If it does not, then download it
  !gdown --id 1IqVy6z09vymy4KJb4AcJtV3Q5hnyFLFB

# Load the dataset
data = pd.read_csv(filename)

# Move important information out of the data variable since pandas does not like to hold on to information like...
 
# Age of people in images
data_ages = np.expand_dims(np.array(data['age']),1) / 1.

# Images of people
print('Breaking up dataset...')
data_images = np.zeros((48,48,len(data)))
for i in range(len(data)):
  data_images[:,:,i] = np.array(data['pixels'][i].split(),'float64').reshape(48,48) / 255.
print('...Done')

Downloading...
From: https://drive.google.com/uc?id=1IqVy6z09vymy4KJb4AcJtV3Q5hnyFLFB
To: /content/age_gender.csv
200MB [00:02, 77.7MB/s]
Breaking up dataset...
...Done


# Deep learning competition
---
##### In the next portion of this notebook, we propose a challenge for you to modify a CNN to get the lowest training time and the most accurate network.


### Adjusting hyperparameters
---
##### The code chunk below has forms that can be modified to change the architecture of the network used to predict the age of the faces in the images. 
##### When you are done modifying the network architecture (this can be done however many times you want), then pressing the "Verify model" will estimate how long it will take to train the model. **_Please keep note that the training time must stay under 20 minutes to qualify for the competition._**
##### To give you an idea on if your network is doing well or not, here is the report generated from a network trained with all of the default settings:
`Time stamp:`

`2021-06-22 16:15:32.275939`

`Network parameters:`

`Conv layer 1: 128 filters of size (3,3).`

`Conv layer 2: 64 filters of size (3,3).`

`Conv layer 3: 32 filters of size (3,3).`

`MLP layer 1: 512 connections.`

`MLP layer 2: 256 connections.`

`MLP layer 3: 64 connections.`

`Network error:`

`The network trained for 20 epochs and took 7.82 minutes to train.`

`The network has an average prediction error of 8.4 years.`

#### Convolutional layers
---

In [40]:
# Generate a model
#@title Number of convolution layers? (at least 1) {run: "auto"}
number_of_conv_layers =  3#@param {type:"integer"}
number_of_conv_layers = max(number_of_conv_layers,1)

style = {'description_width': 'initial'}
default_filters = [128,64,32]
if number_of_conv_layers > 3:
  for i in range(3,number_of_conv_layers):
    default_filters.append(1)
default_size = []
for i in range(number_of_conv_layers):
  default_size.append(3)

conv_layers = widgets.GridspecLayout(number_of_conv_layers, 2, width='600px')
for i in range(number_of_conv_layers):
        conv_layers[i, 0] = widgets.IntText(value=default_filters[i],description='Filters in conv layer '+str(i+1)+':',layout=widgets.Layout(width='70%'),style=style)
        conv_layers[i, 1] = widgets.IntText(value=default_size[i],description='Size of filter in layer '+str(i+1)+':',layout=widgets.Layout(width='70%'),style=style)
display(conv_layers)

GridspecLayout(children=(IntText(value=128, description='Filters in conv layer 1:', layout=Layout(grid_area='w…

#### Multi-layer perceptron 
---


In [41]:
#@title Number of MLP layers? (at least 1, excluding the output layer) {run: "auto"}
number_of_mlp_layers =  3#@param {type:"integer"}
number_of_mlp_layers = max(number_of_mlp_layers,1)

style = {'description_width': 'initial'}
default_connections = [512,256,64]
if number_of_mlp_layers > 3:
  for i in range(3,number_of_mlp_layers):
    default_connections.append(1)

mlp_layers = widgets.GridspecLayout(number_of_conv_layers, 1, width='600px')
for i in range(number_of_conv_layers):
        mlp_layers[i, 0] = widgets.IntText(value=default_connections[i],description='Inputs for MLP layer '+str(i+1)+':',layout=widgets.Layout(width='35%'),style=style)
display(mlp_layers)

GridspecLayout(children=(IntText(value=512, description='Inputs for MLP layer 1:', layout=Layout(grid_area='wi…

#### Training amount
---

In [42]:
#@title Number of training epochs? {run: "auto"}
number_of_epochs =  20 #@param {type:"integer"}
number_of_epochs = max(number_of_epochs,1)

#### Verifying network
---
##### When you have set up your network parameters, then run the next code chunk and click the "Verify model" button. This will double check the selected network parameters.

In [ ]:
#@title --- Hidden code (double-click to show code) ---
# Internal functions that will be used
def create_network(conv_layers,mlp_layers):
  modules = []
  img_size = 48
  current_channels = 1
  
  # Generate convolutional, relu, max pool, and batch norm layers
  count = 0
  for i in range(conv_layers.n_rows):
    number_of_filters = conv_layers.children[count].value
    size_of_filters = conv_layers.children[count+1].value
    max_pool_size = 2
    count += 2
    modules.append(nn.Conv2d(current_channels,number_of_filters,size_of_filters))
    modules.append(nn.ReLU())
    modules.append(nn.MaxPool2d(max_pool_size))
    modules.append(nn.BatchNorm2d(number_of_filters))

    current_channels = number_of_filters
    img_size -= ((size_of_filters//2) * 2)
    img_size //= 2

  # Add on flatten layer
  modules.append(nn.Flatten(start_dim=1,end_dim=-1))
  current_input = (img_size**2) * current_channels
  
  # Add MLP layers
  for i in range(mlp_layers.n_rows):
    number_of_connections = mlp_layers.children[i].value
    modules.append(nn.Linear(int(current_input),number_of_connections))
    modules.append(nn.ReLU())

    current_input = number_of_connections

  # Add output
  modules.append(nn.Linear(current_input,1))

  # Convert list to network
  temp_model = nn.Sequential(*modules)

  return temp_model

def verify_model(b):
  # Generate the model
  b.disabled = True
  print('Generating model...')
  temp_model = create_network(conv_layers,mlp_layers)

  # Try to attach the network to a GPU
  try:
    temp_model = temp_model.cuda()
    canUseGPU = True
  except:
    print('Warning: The GPU has not been enabled. Training may take longer than expected.')
    canUseGPU = False
  print('...Done')

  # Time how long it takes to complete a batch of images
  print('Setting up dataset...')
  dataset = UTKFaceDataset(data_images,data_ages)
  train_dataloader = torch.utils.data.DataLoader(dataset=dataset, shuffle=True, batch_size=batchsize)
  loss = nn.MSELoss()
  optimizer = torch.optim.Adam(temp_model.parameters())
  print('...Done')

  # Estimate the time it will take to train the model by running a batch of the training set 5 times and then averaging
  print('Timing model...')
  est_time = []
  for i in range(5):
    # Start the timer
    begin_time = time.time()

    # Run the training batch
    dataiter = iter(train_dataloader)
    batch = dataiter.next()

    # Move things to GPU
    if canUseGPU:
      batch['image'] = batch['image'].cuda()
      batch['age'] = batch['age'].cuda()

    # Do typical training loop things
    output = temp_model.forward(batch['image'])
    loss_value = loss(output,batch['age'])
    loss_value.backward()
    optimizer.step() 

    # Stop the timer and add the elapsed time to a variable
    end_time = time.time()
    est_time.append((end_time - begin_time) * 1.5 * ((18964//batchsize)+1))
    print('...')

  # Average the time estiamted for an epoch
  est_time_tensor = torch.tensor(est_time)
  est_time_tensor = torch.mean(est_time_tensor[2:-1])
  est_ep_time = est_time_tensor.item()

  # Use that to estimate how long it will take to train
  est_train_time = round((((est_ep_time) * (number_of_epochs+1)) / 60),ndigits=2)
  print('...Done')

  # Print the results
  print('Estimated epoch time: ' + str(est_ep_time) + ' seconds.')
  print('Estimated training time: ' + str(est_train_time) + ' minutes.' )
  if est_train_time > 15:
    print('WARNING: Your estimated training time is high. You may want to consider decreasing some hyperparameters.')
  b.disabled = False

# Actual code
# Fixed parameters
batchsize = 10

# Add in button for verifying model
verify_button = widgets.Button(description="Verify model")
output = widgets.Output()
display(verify_button,output)

# Set up callback
verify_button.on_click(verify_model)

### Training the model
---
##### A required practice for training any type of AI is to break up the entire dataset you want to train the model on into a _training_, _validation_, and _testing_ set. 
##### The _training_ set is used to actually train the model, and usually is approximately 70% to 80% of the whole dataset.
##### That amount of the dataset that is left over typically gets halved, where one half, the _validation_ set, gets used to make sure that the model is not memorizing the _training_ set. Keep in mind that the model is **_not_** trained on.
##### The rest of the dataset gets lumped into the _testing_ set, which is used after the training has been completed to test the model.

In [ ]:
#@title --- Hidden code (double-click to show code) ---
# Define method for updating plot
def update_plots(ep_range,train_history,valid_history):
  clear_output(wait=True)
  if not canUseGPU:
    print('Warning: The GPU has not been enabled. Training may take longer than expected.')

  fig = plt.figure(figsize=(10,10))
  ax1 = fig.add_subplot(2,1,1)
  plt1 = plt.plot(ep_range,train_history,'ro-')
  ax1.set_title('Average training loss');
  ax1.set_xlabel('Epoch');
  ax1.set_ylabel('Average MSE loss');

  ax2 = fig.add_subplot(2,1,2)
  plt2 = plt.plot(ep_range,valid_history,'ro-')
  ax2.set_title('Average validation loss');
  ax2.set_xlabel('Epoch');
  ax2.set_ylabel('Average MSE loss');
  plt.show()

# Import progress bar
from tqdm.notebook import tqdm

# Create the...
# Model
print('Generating model...')
model = create_network(conv_layers,mlp_layers)

# Try to attach the network to a GPU
try:
  model = model.cuda()
  canUseGPU = True
except:
  print('Warning: The GPU has not been enabled. Training may take longer than expected.')
  canUseGPU = False

# Training dataset
training_end = round(len(data_ages)*0.8)
train_dataset = UTKFaceDataset(data_images[:,:,0:training_end], data_ages[0:training_end])
train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=batchsize)

# Validation dataset
validation_end = round(len(data_ages)*0.1)
validation_dataset = UTKFaceDataset(data_images[:,:,training_end:training_end+validation_end], data_ages[training_end:training_end+validation_end])
validation_data_loader = torch.utils.data.DataLoader(dataset=validation_dataset, shuffle=False, batch_size=batchsize)

# Testing dataset
testing_dataset = UTKFaceDataset(data_images[:,:,training_end+validation_end:-1], data_ages[training_end+validation_end:-1])
testing_data_loader = torch.utils.data.DataLoader(dataset=testing_dataset, shuffle=False, batch_size=batchsize)

# Define which loss function we will be using
loss = nn.MSELoss()

# Specify the optimizer (or gradient descent algorithm)
optimizer = torch.optim.Adam(model.parameters())

# Set up plots
ep_range = np.zeros(1)

# Run an initial pass to get base line values
with torch.no_grad():
  epoch_loss = 0
  with tqdm(total=train_dataset.__len__(), desc=f'Epoch {0}/{number_of_epochs}', unit='img') as pbar:
    for batch in train_data_loader:
      # Move things to GPU
      if canUseGPU:
        batch['image'] = batch['image'].cuda()
        batch['age'] = batch['age'].cuda()

      # Perform forward pass
      output = model.forward(batch['image'])
      
      # Calculate loss
      loss_value = loss(output,batch['age'])
      epoch_loss += loss_value.item()
      pbar.set_postfix(**{'current loss': loss_value.item()})

      # Update progress bar
      pbar.update(batch['image'].shape[0])

  train_history = np.array((epoch_loss/train_dataset.__len__())*batchsize)

  with tqdm(total=validation_dataset.__len__(), desc=f'Epoch {0}/{number_of_epochs}', unit='img') as pbar:
    epoch_loss = 0
    for batch in validation_data_loader:
      # Move things to GPU
      if canUseGPU:
        batch['image'] = batch['image'].cuda()
        batch['age'] = batch['age'].cuda()

      # Perform forward pass
      output = model.forward(batch['image'])
      
      # Calculate loss
      loss_value = loss(output,batch['age'])
      epoch_loss += loss_value.item()
      pbar.set_postfix(**{'current loss': loss_value.item()})

      # Update progress bar
      pbar.update(batch['image'].shape[0])

  valid_history = np.array((epoch_loss/validation_dataset.__len__())*batchsize)

# Train the network!
start_time = time.time()
for ep in range(number_of_epochs):
  # Training loop
  update_plots(ep_range,train_history,valid_history)
  epoch_loss = 0

  with tqdm(total=train_dataset.__len__(), desc=f'Epoch {ep + 1}/{number_of_epochs}', unit='img') as pbar:
      for batch in train_data_loader:
        # Move things to GPU
        if canUseGPU:
          batch['image'] = batch['image'].cuda()
          batch['age'] = batch['age'].cuda()

        # Clear the gradients
        optimizer.zero_grad()

        # Perform forward pass
        output = model.forward(batch['image'])
        
        # Calculate loss
        loss_value = loss(output,batch['age'])
        epoch_loss += loss_value.item()
        pbar.set_postfix(**{'current loss': loss_value.item()})

        # Back-propogate
        loss_value.backward()

        # Gradient descent
        optimizer.step()  
        
        # Update progress bar
        pbar.update(batch['image'].shape[0])
      pbar.set_postfix(**{'average training loss': (epoch_loss/train_dataset.__len__())*batchsize})

      # Update training plot
      ep_range = np.append(ep_range,ep+1)
      train_history = np.append(train_history,(epoch_loss/train_dataset.__len__())*batchsize)
      #update_plots(ep_range,train_history,valid_history)
      
  # Validation loop
  epoch_loss = 0
  with tqdm(total=validation_dataset.__len__(), desc=f'Validation', unit='img') as pbar:
      for batch in validation_data_loader:
        with torch.no_grad():
          # Move things to GPU
          if canUseGPU:
            batch['image'] = batch['image'].cuda()
            batch['age'] = batch['age'].cuda()

          # Perform forward pass
          output = model.forward(batch['image'])
          
          # Calculate loss
          loss_value = loss(output,batch['age'])
          epoch_loss += loss_value.item()
          pbar.set_postfix(**{'current loss': loss_value.item()})
          
          # Update progress bar
          pbar.update(batch['image'].shape[0])
      pbar.set_postfix(**{'average validation loss': (epoch_loss/validation_dataset.__len__())*batchsize}) 
      
      # Update validation plot
      valid_history = np.append(valid_history,(epoch_loss/train_dataset.__len__())*batchsize)
      update_plots(ep_range,train_history,valid_history)

end_time = time.time()
training_time = (end_time-start_time)/60
print('The training took ' + str(round((end_time-start_time)/60,2)) + ' minutes to complete.')

### Predicting the age from the testing set
---
##### In the following code chunk, the model that was just trained is used to predict the age of the _testing_ portion of the dataset.
##### The text output from the following code chunk specifies the average prediction error and how long it takes the network to predict the age of 10 images. You should compare these values to how well you did in the previous notebook at predicting ages. _(Keep in mind that the time the network takes to predict the age is outputted in milliseconds)_

In [ ]:
#@title --- Hidden code (double-click to show code) ---
# Testing set
model.eval()

epoch_loss = 0
with tqdm(total=testing_dataset.__len__(), desc=f'Testing', unit='img') as pbar:
    for batch in testing_data_loader:
      with torch.no_grad():
        # Move things to GPU
        if canUseGPU:
          batch['image'] = batch['image'].cuda()
          batch['age'] = batch['age'].cuda()

        # Perform forward pass
        output = model.forward(batch['image'])
        
        # Calculate loss
        loss_value = loss(output,batch['age'])
        epoch_loss += loss_value.item()
        pbar.set_postfix(**{'current loss': loss_value.item()})

        # Update progress bar
        pbar.update(batch['image'].shape[0])
    pbar.set_postfix(**{'average testing loss': (epoch_loss/testing_dataset.__len__())}) 

# Time 10 images
start_time = time.time()
output = model.forward(batch['image'])
end_time = time.time()
prediction_error = (epoch_loss/testing_dataset.__len__())**0.5

# Print out some stats
print('Your final average testing loss is: ' + str(epoch_loss/testing_dataset.__len__()))
print('Your network has an average prediction error of ' + str(round(prediction_error,2))  + ' years.')
print('The model takes ' + str(round((end_time-start_time)*1000,2)) + ' milliseconds to predict the age of 10 images.')
print('Here are some images and their estimated age:')

# Get a random set of 15 images
random_test_ind = np.random.randint(0,testing_dataset.__len__(),15)

# Display the images
fig = plt.figure(figsize=(23,23))
for i in range(15):
  with torch.no_grad():
    batch = testing_dataset.__getitem__(random_test_ind[i])

    # Move things to GPU
    if canUseGPU:
      batch['image'] = batch['image'].cuda()
      batch['age'] = batch['age'].cuda()

    # Perform forward pass
    output = model.forward(batch['image'].unsqueeze(0)) 
    
  # Show the image
  fig.add_subplot(3,5,i+1)
  plt.imshow(batch['image'].squeeze(0).cpu().detach().numpy(), cmap='gray')
  plt.gca().set_title("Predicted: " + str(np.round(output[0,0].cpu().detach().numpy())) + " | Actual: " + str(batch['age'][0].cpu().detach().numpy()))
plt.gcf().subplots_adjust(bottom=0.4)

# Generate report
---
##### To submit your network for the competition, run the following two code chunks. You will have to use the output from the first code chunk to fill in the form that is outputted from the second code chunk. 
##### You are welcome to submit as many times as you would like!
#####**Note: Please fill in the form _exactly_ as it is outputted from the first code chunk. Submissions that are not filled in correctly will _not_ be considered!**

In [ ]:
#@title --- Hidden code (double-click to show code) ---
count = 0
filters = []
filter_size = []
for i in range(conv_layers.n_rows):
  filters.append(conv_layers.children[count].value)
  filter_size.append(conv_layers.children[count+1].value)
  count += 2

print("Number of convolution filters: " + str(len(filters)))
print("Number of filters in convolution layers: " + str(filters))
print("Size of filters: " + str(filter_size))

connections = []
for i in range(mlp_layers.n_rows):
  connections.append(mlp_layers.children[i].value)

print("Number of connections in MLP: " + str(connections))
print("Number of epochs: " + str(number_of_epochs))
print("Time it took to train: " + str(round(training_time,5)))
print("Prediction error: " + str(round(prediction_error,5)))

In [59]:
#@title --- Hidden code (double-click to show code) ---
%%html
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="description" content="Submission form">
  <title>Submission Form</title>
</head>

<body>
  <h2 class="content-head is-center"> 
    <p> CNN competition submission form. </p> 
    Please fill in this form by copying the information from the previous code chunk output.
  </h2>

<!-- START HERE -->
   <link rel="stylesheet" href="https://unpkg.com/purecss@1.0.0/build/pure-min.css">
   <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.4.0/css/font-awesome.min.css">
   <!-- Style The Contact Form How Ever You Prefer -->
   <link rel="stylesheet" href="style.css">

  <form class="gform pure-form pure-form-stacked" method="POST" data-email="example@email.net"
  action="https://script.google.com/macros/s/AKfycbxIIZDYlT-PSTYeWHCnUJT0ZlrrNzbzXnC3pZFg-rrMeAYnNFvS/exec">
    <!-- change the form action to your script url -->

    <div class="form-elements">
      <fieldset class="pure-group">
        <label for="name">Name:</label>
        <input id="name" name="Name" />
      </fieldset>

      <fieldset class="pure-group">
        <label for="number_of_conv_layers">Number of convolution layers:</label>
        <input id="number_of_conv_layers" name="Number of convolution layers" />
      </fieldset>

      <fieldset class="pure-group">
        <label for="number_of_conv_filters">Number of filters in convolution layers:</label>
        <input id="number_of_conv_filters" name="Number of filters per layer" />
      </fieldset>

      <fieldset class="pure-group">
        <label for="filter_size">Size of filters:</label>
        <input id="filter_size" name="Size of filters" />
      </fieldset>

      <fieldset class="pure-group">
        <label for="number_of_connections">Number of connections in MLP:</label>
        <input id="number_of_connections" name="Number of MLP connections" />
      </fieldset>

      <fieldset class="pure-group">
        <label for="number_of_epochs">Number of epochs:</label>
        <input id="number_of_epochs" name="Number of training epochs" />
      </fieldset>

      <fieldset class="pure-group">
        <label for="training_time">Time it took to train:</label>
        <input id="training_time" name="Training time [minutes]" />
      </fieldset>

      <fieldset class="pure-group">
        <label for="prediction_error">Prediction error:</label>
        <input id="prediction_error" name="Prediction error [years]" />
      </fieldset>

      <fieldset class="pure-group honeypot-field">
        <label for="honeypot">Keep this field blank</label>
        <input id="honeypot" type="text" name="honeypot" value="" />
      </fieldset>

      <button class="button-success pure-button button-xlarge">
        <i class="fa fa-paper-plane"></i>&nbsp;Send</button>
    </div>

  </form>
<!-- END -->

</body>
</html>